In [1]:
import mlflow
import os
from dotenv import load_dotenv
from mlflow.tracking import MlflowClient

# Step 2: Set up the MLflow tracking server
load_dotenv()
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))

In [2]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from imblearn.over_sampling import ADASYN

In [3]:
df = pd.read_csv('D:\datascience\campusx\PROJECTS\Second_project\cleaned_data.csv',index_col=0).dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [4]:
mlflow.set_experiment("EXP_5-Xgboost_hpt_1000")


<Experiment: artifact_location='s3://apoorv-mlfow-bucket/285964941007082429', creation_time=1758734578723, experiment_id='285964941007082429', last_update_time=1758734578723, lifecycle_stage='active', name='EXP_5-Xgboost_hpt_1000', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [5]:


# Step 1: Remove rows with NaN labels
df = df.dropna(subset=['category'])

X_raw = df['clean_comment']
y_raw = df['category']

# Step 2: Train-test split first
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_raw, y_raw, test_size=0.2, random_state=42, stratify=y_raw
)

# Step 3: TF-IDF vectorizer fit on training data only
ngram_range = (1, 3)
max_features = 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train_raw)
X_test_vec = vectorizer.transform(X_test_raw)

# Step 4: Apply ADASYN only on training set
adasyn = ADASYN(random_state=42)
X_train_vec_resampled, y_train_resampled = adasyn.fit_resample(X_train_vec, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, best_params=None):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_ADASYN_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        mlflow.log_param("algo_name", model_name)

        if best_params:
            for param, value in best_params.items():
                mlflow.log_param(param, value)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        mlflow.sklearn.log_model(model, f"{model_name}_model")

# Step 5: Optuna objective function for KNN
def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 30)
    p = trial.suggest_categorical('p', [1, 2])

    model = KNeighborsClassifier(n_neighbors=n_neighbors, p=p)
    return accuracy_score(y_test, model.fit(X_train_vec_resampled, y_train_resampled).predict(X_test_vec))

# Step 6: Run Optuna for KNN, log the best model & parameters
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_knn, n_trials=30)

    best_params = study.best_params
    print("Best parameters:", best_params)
    print("Best accuracy:", study.best_value)

    best_model = KNeighborsClassifier(
        n_neighbors=best_params['n_neighbors'], 
        p=best_params['p']
    )

    log_mlflow(
        "KNN", 
        best_model, 
        X_train_vec_resampled, 
        X_test_vec, 
        y_train_resampled, 
        y_test, 
        best_params=best_params
    )

# Run the experiment
run_optuna_experiment()


[I 2025-09-25 12:45:43,334] A new study created in memory with name: no-name-93d6a488-9e44-4e84-bc62-df966a0dea38
[I 2025-09-25 12:45:49,517] Trial 0 finished with value: 0.37256238919950907 and parameters: {'n_neighbors': 24, 'p': 1}. Best is trial 0 with value: 0.37256238919950907.
[I 2025-09-25 12:45:52,472] Trial 1 finished with value: 0.3758352652393291 and parameters: {'n_neighbors': 21, 'p': 1}. Best is trial 1 with value: 0.3758352652393291.
[I 2025-09-25 12:46:00,887] Trial 2 finished with value: 0.42438292649665893 and parameters: {'n_neighbors': 27, 'p': 2}. Best is trial 2 with value: 0.42438292649665893.
[I 2025-09-25 12:46:03,850] Trial 3 finished with value: 0.37815355243420157 and parameters: {'n_neighbors': 15, 'p': 1}. Best is trial 2 with value: 0.42438292649665893.
[I 2025-09-25 12:46:12,183] Trial 4 finished with value: 0.4794763398336288 and parameters: {'n_neighbors': 7, 'p': 2}. Best is trial 4 with value: 0.4794763398336288.
[I 2025-09-25 12:46:20,672] Trial 5 

Best parameters: {'n_neighbors': 7, 'p': 2}
Best accuracy: 0.4794763398336288


2025/09/25 12:50:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/25 12:50:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run KNN_ADASYN_TFIDF_Trigrams at: http://ec2-16-171-17-119.eu-north-1.compute.amazonaws.com:5000/#/experiments/285964941007082429/runs/e862a2124bb143398dbffc089c250368
🧪 View experiment at: http://ec2-16-171-17-119.eu-north-1.compute.amazonaws.com:5000/#/experiments/285964941007082429
